In [1]:
%%writefile ../crypto_chaotic/audioServer.py
#!/usr/bin/env python

# Copyright (c) Twisted Matrix Laboratories.
# See LICENSE for details.

from twisted.internet.protocol import DatagramProtocol
from twisted.internet import reactor

# Here's a UDP version of the simplest possible protocol
class EchoUDP(DatagramProtocol):
    def datagramReceived(self, datagram, address):
        #print datagram
        self.transport.write(datagram, address)

def main():
    reactor.listenUDP(8000, EchoUDP())
    reactor.run()

if __name__ == '__main__':
    main()

Overwriting ../crypto_chaotic/audioServer.py


In [1]:
%%bash
pushd ../crypto_chaotic
python2 -m audioServer

/run/media/abhishek/bf2b188b-08b7-4520-8562-2d8b267dc5ce/2016/crypto-chaotic/crypto_chaotic /run/media/abhishek/bf2b188b-08b7-4520-8562-2d8b267dc5ce/2016/crypto-chaotic/doc


Traceback (most recent call last):
  File "/usr/lib64/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib64/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/run/media/abhishek/bf2b188b-08b7-4520-8562-2d8b267dc5ce/2016/crypto-chaotic/crypto_chaotic/audioServer.py", line 20, in <module>
    main()
  File "/run/media/abhishek/bf2b188b-08b7-4520-8562-2d8b267dc5ce/2016/crypto-chaotic/crypto_chaotic/audioServer.py", line 16, in main
    reactor.listenUDP(8000, EchoUDP())
  File "/usr/lib64/python2.7/site-packages/twisted/internet/posixbase.py", line 373, in listenUDP
    p.startListening()
  File "/usr/lib64/python2.7/site-packages/twisted/internet/udp.py", line 101, in startListening
    self._bindSocket()
  File "/usr/lib64/python2.7/site-packages/twisted/internet/udp.py", line 109, in _bindSocket
    raise error.CannotListenError, (self.interface, self.port, le)
twisted.internet.error.CannotListenE